In [ ]:
from datetime import datetime
import holidays

# Fonction pour obtenir le jour de la semaine et la semaine de l'année à partir d'une date
def get_weekday_and_week(date_str):
    date = datetime.strptime(date_str, '%Y-%m-%d')
    weekday = date.strftime('%A')  # Jour de la semaine complet (en anglais)
    week_number = date.strftime('%W')  # Numéro de semaine dans l'année
    return weekday, int(week_number)

# Exemple d'utilisation
date_str = '2024-07-12'
weekday, week_number = get_weekday_and_week(date_str)

print(f"Pour la date {date_str}:")
print(f"Jour de la semaine: {weekday}")
print(f"Semaine de l'année: semaine {week_number + 1}")  # Ajouter 1 car les semaines commencent à 0


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Chargement des données et prétraitement (vous devez avoir déjà ces étapes prêtes)
df = pd.read_csv('../data_processed/traffic_shift_nord.csv')

# Créer une colonne de date
df['date'] = pd.to_datetime(df['annee'].astype(str) + '-' + df['mois'].astype(str) + '-' + df['jour'].astype(str))
df=df.dropna()

In [ ]:


# Prétraitement des données
X = df[['Identifiant_arc', 'C_AR', 'AM/PM', 'annee', 'mois', 'jour', 'vacances', 'semaine']]
y = df['Etat_trafic']

# Transformer les variables catégorielles avec OneHotEncoder
cat_features = ['C_AR', 'AM/PM']
numeric_features = ['Identifiant_arc', 'annee', 'mois', 'jour', 'vacances', 'semaine']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), cat_features),
        ('num', 'passthrough', numeric_features)
    ])

# Appliquer le prétraitement
X_preprocessed = preprocessor.fit_transform(X)

# Entrainement du modèle RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_preprocessed, y)

# Fonction pour prédire la moyenne de l'état du trafic pour une date donnée
def predict_traffic_state(date_str):
    weekday, week_number = get_weekday_and_week(date_str)
    
    # Supposons que nous voulons prédire pour tous les arrondissements
    unique_arcs = df['Identifiant_arc'].unique()
    
    predictions = []
    for arc in unique_arcs:
        for period in ['AM', 'PM']:
            # Préparer les données pour la prédiction
            prediction_data = {
                'Identifiant_arc': arc,
                'C_AR': df[df['Identifiant_arc'] == arc]['C_AR'].iloc[0],  # Prendre la première valeur trouvée
                'AM/PM': period,
                'annee': int(date_str[:4]),  # Extraire l'année de la date
                'mois': int(date_str[5:7]),  # Extraire le mois de la date
                'jour': int(date_str[8:10]),  # Extraire le jour de la date
                'vacances': 1 if date_str in holidays.France() else 0,  # Vérifier si c'est un jour férié
                'semaine': week_number
            }
            
            # Convertir en DataFrame pour la prédiction
            prediction_df = pd.DataFrame([prediction_data])
            
            # Appliquer le même prétraitement que celui utilisé pour l'entraînement
            prediction_df_preprocessed = preprocessor.transform(prediction_df)
            
            # Faire la prédiction
            prediction = model.predict(prediction_df_preprocessed)
            predictions.append({
                'Identifiant_arc': arc,
                'AM/PM': period,
                'Etat_trafic_mean': prediction[0]
            })
    
    return predictions

# Exemple d'utilisation pour la date '2024-07-12'
predictions = predict_traffic_state('2024-07-12')
print(predictions)


In [59]:
import pandas as pd
# Convert additional data to DataFrame
df_additional = pd.DataFrame(predictions)
# Utilisation de pivot pour transformer le DataFrame
df_transformed = df_additional.pivot(index='Identifiant_arc', columns='AM/PM', values='Etat_trafic_mean').reset_index()

# Renommer les colonnes si nécessaire
df_transformed.columns.name = None  # Retirer le nom de la colonne index
# Arrondir les valeurs à 1 décimale après la virgule
df_transformed['AM'] = df_transformed['AM'].round(1)
df_transformed['PM'] = df_transformed['PM'].round(1)
# Afficher le DataFrame transformé
print(df_transformed)

     Identifiant_arc   AM   PM
0                256  1.1  1.0
1                527  1.8  1.4
2                528  1.7  1.4
3                547  2.6  2.2
4                548  3.0  2.6
..               ...  ...  ...
130             6557  2.7  2.3
131             6558  1.6  1.4
132             6559  1.6  1.4
133             6670  2.3  2.7
134             6742  1.0  1.0

[135 rows x 3 columns]


In [62]:
df_arc=pd.read_csv('../data_processed/traffic_data_geo.csv')


In [77]:
df_additional = pd.DataFrame(predictions)
# Utilisation de pivot pour transformer le DataFrame
df_transformed = df_additional.pivot(index='Identifiant_arc', columns='AM/PM', values='Etat_trafic_mean').reset_index()

# Renommer les colonnes si nécessaire
df_transformed.columns.name = None  # Retirer le nom de la colonne index
# Arrondir les valeurs à 1 décimale après la virgule
df_transformed['AM'] = df_transformed['AM'].round(1)
df_transformed['PM'] = df_transformed['PM'].round(1)

df_result_maps = pd.merge(df_transformed, df_arc, left_on='Identifiant_arc', right_on='Identifiant arc', how='inner')

# Sélectionner les colonnes nécessaires pour df_result_maps
df_result_maps = df_result_maps[['Identifiant_arc', 'AM', 'PM', 'Libelle', 'latitude', 'longitude']]
# Afficher df_result_maps
print("DataFrame résultant après merge avec df_arc :")
print(df_result_maps)
df_result_maps = df_result_maps.drop_duplicates(subset=['Identifiant_arc', 'AM', 'PM', 'Libelle', 'latitude', 'longitude'], keep='first')

from branca.colormap import LinearColormap
# Création de la carte pour AM
mymap_am = folium.Map(location=[df_result_maps['latitude'].iloc[0], df_result_maps['longitude'].iloc[0]], zoom_start=13)

# Création de la carte pour PM
mymap_pm = folium.Map(location=[df_result_maps['latitude'].iloc[0], df_result_maps['longitude'].iloc[0]], zoom_start=13)

# Création d'une colormap pour AM
colormap_am = LinearColormap(['green', 'yellow', 'red'], vmin=df_result_maps['AM'].min(), vmax=df_result_maps['AM'].max())
colormap_am.caption = 'Traffic AM'

# Création d'une colormap pour PM
colormap_pm = LinearColormap(['green', 'yellow', 'red'], vmin=df_result_maps['PM'].min(), vmax=df_result_maps['PM'].max())
colormap_pm.caption = 'Traffic PM'

# Ajout des marqueurs pour chaque station sur la carte AM
for index, row in df_result_maps.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=10,
        popup=row['Libelle'],
        tooltip=f"AM: {row['AM']}",
        color=colormap_am(row['AM']),
        fill=True,
        fill_opacity=0.7,
    ).add_to(mymap_am)

# Ajout des marqueurs pour chaque station sur la carte PM
for index, row in df_result_maps.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=10,
        popup=row['Libelle'],
        tooltip=f"PM: {row['PM']}",
        color=colormap_pm(row['PM']),
        fill=True,
        fill_opacity=0.7,
    ).add_to(mymap_pm)

# Ajout de la légende des couleurs pour AM
colormap_am.add_to(mymap_am)

# Ajout de la légende des couleurs pour PM
colormap_pm.add_to(mymap_pm)

# Sauvegarde des cartes
mymap_am.save('carte_traffic_AM.html')
mymap_pm.save('carte_traffic_PM.html')

print("Cartes générées avec succès. Ouvrez 'carte_traffic_AM.html' et 'carte_traffic_PM.html' dans votre navigateur.")


DataFrame résultant après merge avec df_arc :
     Identifiant_arc   AM   PM               Libelle   latitude  longitude
0                256  1.1  1.0      Bd_des_Capucines  48.870150   2.328894
1                527  1.8  1.4      Bd_de_Strasbourg  48.869953   2.354587
2                527  1.8  1.4      Bd_de_Strasbourg  48.869953   2.354587
3                528  1.7  1.4  Rue_du_Fbg_St_Martin  48.870413   2.356351
4                528  1.7  1.4  Rue_du_Fbg_St_Martin  48.870413   2.356351
..               ...  ...  ...                   ...        ...        ...
262             6559  1.6  1.4            La_Fayette  48.878797   2.354930
263             6670  2.3  2.7         Aubervilliers  48.896117   2.370886
264             6670  2.3  2.7         Aubervilliers  48.896117   2.370886
265             6742  1.0  1.0          Bd_St_Martin  48.868138   2.361370
266             6742  1.0  1.0          Bd_St_Martin  48.868138   2.361370

[267 rows x 6 columns]
Cartes générées avec succès. O

In [ ]:
predictions = predict_traffic_state('2024-07-12')
print(predictions)

In [78]:
import pandas as pd
import folium
from branca.colormap import LinearColormap

def generate_traffic_maps(predictions, df_arc, file_am='carte_traffic_AM.html', file_pm='carte_traffic_PM.html'):
    # Utilisation de pivot pour transformer le DataFrame
    df_transformed = predictions.pivot(index='Identifiant_arc', columns='AM/PM', values='Etat_trafic_mean').reset_index()

    # Arrondir les valeurs à 1 décimale après la virgule
    df_transformed['AM'] = df_transformed['AM'].round(1)
    df_transformed['PM'] = df_transformed['PM'].round(1)

    # Fusion avec df_arc
    df_result_maps = pd.merge(df_transformed, df_arc, left_on='Identifiant_arc', right_on='Identifiant arc', how='inner')

    # Sélectionner les colonnes nécessaires pour df_result_maps
    df_result_maps = df_result_maps[['Identifiant_arc', 'AM', 'PM', 'Libelle', 'latitude', 'longitude']]
    df_result_maps = df_result_maps.drop_duplicates(subset=['Identifiant_arc', 'AM', 'PM', 'Libelle', 'latitude', 'longitude'], keep='first')

    # Création des cartes pour AM et PM
    mymap_am = create_traffic_map(df_result_maps, 'AM', 'Traffic AM', file_am)
    mymap_pm = create_traffic_map(df_result_maps, 'PM', 'Traffic PM', file_pm)

    print(f"Cartes générées avec succès. Ouvrez '{file_am}' et '{file_pm}' dans votre navigateur.")

def create_traffic_map(df_result_maps, time_period, caption, file_name):
    # Création de la carte
    mymap = folium.Map(location=[df_result_maps['latitude'].iloc[0], df_result_maps['longitude'].iloc[0]], zoom_start=13)

    # Création d'une colormap
    colormap = LinearColormap(['green', 'yellow', 'red'], vmin=df_result_maps[time_period].min(), vmax=df_result_maps[time_period].max())
    colormap.caption = caption

    # Ajout des marqueurs pour chaque station
    for index, row in df_result_maps.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=10,
            popup=row['Libelle'],
            tooltip=f"{caption}: {row[time_period]}",
            color=colormap(row[time_period]),
            fill=True,
            fill_opacity=0.7,
        ).add_to(mymap)

    # Ajout de la légende des couleurs
    colormap.add_to(mymap)

    # Sauvegarde de la carte
    mymap.save(file_name)

    return mymap  # Retourne la carte créée pour référence si nécessaire


In [80]:
generate_traffic_maps(predict_traffic_state('2024-07-14'),df_arc,file_am='carte_traffic_AM_1407.html', file_pm='carte_traffic_PM_1407.html')

AttributeError: 'list' object has no attribute 'pivot'